In [1]:
import pandas as pd
import numpy as np
import scipy.stats
import statsmodels.stats.multitest
import operator
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats
import gseapy as gp
import re
import sys 

import cptac
import cptac.utils as u

/Users/hannahboekweg/anaconda3/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
en = cptac.Endometrial()
col = cptac.Colon()
br = cptac.Brca()

In [3]:
def wrap_ttest(df, label_column, comparison_columns=None, alpha=.05, return_all=False, correction_method='bonferroni'):
    try:
        '''Verify precondition that label column exists and has exactly 2 unique values'''
        label_values = df[label_column].unique()
        if len(label_values) != 2:
            print("Incorrectly Formatted Dataframe! Label column must have exactly 2 unique values.")
            return None
        
        '''Partition dataframe into two sets, one for each of the two unique values from the label column'''
        partition1 = df.loc[df[label_column] == label_values[0]]
        partition2 = df.loc[df[label_column] == label_values[1]]

        '''If no comparison columns specified, use all columns except the specified labed column'''
        if not comparison_columns:
            comparison_columns = list(df.columns)
            comparison_columns.remove(label_column)

        '''Determine the number of real valued columns on which we will do t-tests'''
        number_of_comparisons = len(comparison_columns)

        '''Store comparisons and p-values in two arrays'''
        comparisons = []
        pvals = []
        
        '''Loop through each comparison column, perform the t-test, and record the p-val'''
        for column in comparison_columns:
            stat, pval = scipy.stats.ttest_ind(partition1[column].dropna(axis=0), partition2[column].dropna(axis=0))
            comparisons.append(column)
            pvals.append(pval)
            
        '''Correct for multiple testing to determine if each comparison meets the new cutoff'''
        results = statsmodels.stats.multitest.multipletests(pvals=pvals, alpha=alpha, method=correction_method)
        reject = results[0]

        '''Format results in a pandas dataframe'''
        results_df = pd.DataFrame(columns=['Comparison','P_Value'])

        '''If return all, add all comparisons and p-values to dataframe'''
        if return_all:
            results_df['Comparison'] = comparisons
            results_df['P_Value'] = pvals

            '''Else only add significant comparisons'''
        else:
            for i in range(0, len(reject)):
                if reject[i]:
                    results_df = results_df.append({'Comparison':comparisons[i],'P_Value':pvals[i]}, ignore_index=True)


        '''Sort dataframe by ascending p-value'''
        results_df = results_df.sort_values(by='P_Value', ascending=True)
        results_df = results_df.reset_index(drop=True)

        '''If results df is not empty, return it, else return None'''
        if len(results_df) > 0:
            return results_df
        else:
            return None


    except:
        print("Incorrectly Formatted Dataframe!")
        return None

In [4]:
mut = en.get_genotype_all_vars("PIK3CA", mutation_hotspot=['E542K', 'E545K', 'H1047R'])
prot = en.get_proteomics(tissue_type="tumor")

cptac warning: In joining the somatic_mutation table, no mutations were found for the following samples, so they were filled with Wildtype_Tumor or Wildtype_Normal: 48 samples for the PIK3CA gene (/Users/hannahboekweg/anaconda3/lib/python3.7/site-packages/cptac/dataset.py, line 311)


In [5]:
joined = mut.join(prot)

en_hotspot_df = joined[joined.Location.str.contains('E542K') | 
                    joined.Location.str.contains('E545K') |
                    joined.Location.str.contains('H1047R')]

# en_hotspot_df

In [6]:
wildtype  = joined.loc[joined.Mutation == "Wildtype_Tumor"]

endo_hotspot = en_hotspot_df.append(wildtype)

prot_and_mutations = endo_hotspot.drop(columns = ["Mutation_Status", "Location"])
# prot_and_mutations

In [7]:
label_values = prot_and_mutations["Mutation"].unique()

'''Partition dataframe into two sets, one for each of the two unique values from the label column'''
partition1 = prot_and_mutations.loc[prot_and_mutations["Mutation"] == label_values[0]]
partition2 = prot_and_mutations.loc[prot_and_mutations["Mutation"] == label_values[1]]

'''If no comparison columns specified, use all columns except the specified labed column'''
comparison_columns = list(prot_and_mutations.columns)
comparison_columns.remove("Mutation")

number_of_comparisons = len(comparison_columns)

'''Store comparisons and p-values in two arrays'''
comparisons = []
pvals = []

'''Loop through each comparison column, perform the t-test, and record the p-val'''
times_through = 0
for column in comparison_columns:  
    times_through += 1
    if len(partition1[column].dropna(axis=0)) <= 1:
#         comparison_columns.remove(column)
        continue
    elif len(partition2[column].dropna(axis=0)) <= 1:
#         comparison_columns.remove(column)
        continue
    else:
        stat, pval = scipy.stats.ttest_ind(partition1[column].dropna(axis=0), partition2[column].dropna(axis=0))
        comparisons.append(column)
        pvals.append(pval)
    
'''Correct for multiple testing to determine if each comparison meets the new cutoff'''
results = statsmodels.stats.multitest.multipletests(pvals=pvals, alpha=.05, method='fdr_bh')
reject = results[0]


'''Format results in a pandas dataframe'''
results_df = pd.DataFrame(columns=['Comparison','P_Value'])

for i in range(0, len(reject)):
    if reject[i]:
        results_df = results_df.append({'Comparison':comparisons[i],'P_Value':pvals[i]}, ignore_index=True)


'''Sort dataframe by ascending p-value'''
results_df = results_df.sort_values(by='P_Value', ascending=True)
results_df = results_df.reset_index(drop=True)

In [8]:
results_df

,Comparison,P_Value


In [9]:
min(results[1])

0.9937396811758431

In [10]:
min(pvals)

0.0006445848058977635

In [11]:
import altair as alt
alt.data_transformers.disable_max_rows()
alt.data_transformers.enable('json')
alt.Chart(pd.DataFrame({"p_val":pvals})).mark_bar().encode(
    x=alt.X("p_val:Q",bin=alt.Bin(step=0.05)),
    y="count()"
)

alt.Chart(...)